> Objetivo: Corrigir os tipos de dados das colunas, inferschema desabilitado para o propósito da ATV

In [0]:
def alterar_tipo(df, coluna, tipo):
    return df.withColumn(coluna, circuitocsv[coluna].cast(tipo))

In [0]:
user = str(dbutils.fs.ls("file:/Workspace/Users/")).split('/')[3]

filespath = "file:/Workspace/Users/" + f"{user}/treinamento_formula1/landing"
savepath = "dbfs:/mnt/formula1/bronze/circuit"
circuitocsv = spark.read.csv(filespath+"/circuito.csv", header=True)

print(user)
print('\nAlterando colunas....')

table_columns = circuitocsv.columns
table_types = ["integer","string","string","string","string","double","double","integer","string"]
table_columntype = dict()

for i in range(len(table_columns)):
    table_columntype[i] = (table_columns[i], table_types[i])

for i in range(len(table_columns)):
    circuitocsv = alterar_tipo(circuitocsv,table_columntype[i][0], table_columntype[i][1])

circuitocsv.printSchema()


print('Criando pasta no dbfs...')
dbutils.fs.mkdirs(savepath)

if(str(dbutils.fs.ls(savepath)).find('circuitoRaw.csv') == -1):
    print('Criando tabela...')
    circuitocsv.write.format('delta').save(f'{savepath}/circuitoRaw.delta', header=True)
else:
    print('Arquivo já existe, sobrescrevendo...')
    circuitocsv.write.format('delta').mode('overwrite').save(f'{savepath}/circuitoRaw.delta', header=True)